In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
df = pd.read_csv("C:/Users/jwaaa/Downloads/KFUPM-Grads-2010-2018/KFUPM_grads_2010_2018.csv", encoding="utf-8")
df.head()
df
#h

,degree,major,honour,id_year,graduated
0,بكالوريوس,الهندسة الميكانيكية التطبيقية,2,2014,2018
1,بكالوريوس,الهندسة المدنية التطبيقية,-1,2013,2018
2,بكالوريوس,العمارة,-1,2013,2018
3,بكالوريوس,الهندسة الكهربائية,2,2014,2018
4,بكالوريوس,الهندسة المعمارية,2,2014,2018
...,...,...,...,...,...
12617,بكالوريوس,المالية,3,2005,2010
12618,بكالوريوس,الهندسة المدنية,-1,2005,2010
12619,بكالوريوس,الهندسة الميكانيكية التطبيقية,-1,2005,2010
12620,ماجستير,الهندسة الكهربائية,-1,1998,2010


In [20]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12622 entries, 0 to 12621
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   degree     12622 non-null  object
 1   major      12622 non-null  object
 2   honour     12622 non-null  int64 
 3   id_year    12622 non-null  int64 
 4   graduated  12622 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 493.2+ KB


,honour,id_year,graduated
count,12622.000000,12622.000000,12622.000000
mean,-0.036365,2008.862225,2014.070987
std,1.540523,3.740825,2.471298
min,-1.000000,1973.000000,2010.000000
25%,-1.000000,2007.000000,2012.000000
50%,-1.000000,2009.000000,2014.000000
75%,2.000000,2011.000000,2016.000000
max,3.000000,2019.000000,2018.000000


In [26]:
# years to graduate
df["years_to_graduate"] = df["graduated"] - df["id_year"]
print(df["years_to_graduate"])

0         4
1         5
2         5
3         4
4         4
         ..
12617     5
12618     5
12619     5
12620    12
12621     5
Name: years_to_graduate, Length: 12622, dtype: int64


In [25]:
HONOR_LABELS = {
    -1: "No honor",
     1: "First honor",
     2: "Second honor",
     3: "Third honor"
}

GPA_RULES = [
    (3.75, "First honor"),
    (3.50, "Second honor"),
    (3.25, "Third honor"),
    (0.00, "No honor"),
]

avg_years_tbl = (
    df.groupby(["degree", "major"], as_index=False)["years_to_graduate"]
      .mean()
      .rename(columns={"years_to_graduate": "avg_years_to_graduate"})
)

honor_dist_tbl = (
    df.groupby(["degree", "major"])["honour"]
      .value_counts(normalize=True) 
        .rename("prop")
        .reset_index()
)
   
counts_tbl = (
    df.groupby(["degree", "major"], as_index=False)
      .size()
      .rename(columns={"size": "n_students"})
) 

summary_tbl = avg_years_tbl.merge(counts_tbl, on=["degree", "major"], how="left")
